## Análisis de la red de tiendas RetailNow

Este cuaderno integra la carga, limpieza y análisis estadístico de ventas, inventarios y satisfacción para apoyar las decisiones sobre la red de tiendas.

### Importación de librerías

Se importan las librerías requeridas para las operaciones numéricas y el análisis de datos.

In [155]:
import numpy as np

In [156]:
import pandas as pd

### Carga y limpieza de datos

Se leen los archivos con rutas absolutas y se eliminan filas nulas para trabajar solo con registros completos.

In [157]:
from pathlib import Path

DATA_DIR = Path("/workspace")

ventas = pd.read_csv(DATA_DIR / "sales.csv").dropna()
inventarios = pd.read_csv(DATA_DIR / "inventories.csv").dropna()
satisfaccion = pd.read_csv(DATA_DIR / "satisfaction.csv").dropna()

### Análisis de ventas

Se calcula el importe total por transacción, así como los ingresos agregados por tienda y producto.

In [158]:
unidades_tienda_producto = (
    ventas.groupby(['ID_Tienda', 'Producto'], as_index=False)['Cantidad_Vendida']
         .sum()
         .rename(columns={'Cantidad_Vendida': 'Unidades_Vendidas'})
)

unidades_por_tienda = (
    ventas.groupby('ID_Tienda', as_index=False)['Cantidad_Vendida']
         .sum()
         .rename(columns={'Cantidad_Vendida': 'Unidades_Vendidas'})
)

unidades_por_producto = (
    ventas.groupby('Producto', as_index=False)['Cantidad_Vendida']
         .sum()
         .rename(columns={'Cantidad_Vendida': 'Unidades_Vendidas'})
)

In [159]:
unidades_tienda_producto


,ID_Tienda,Producto,Unidades_Vendidas
0,1,Producto A,20
1,1,Producto B,15
2,2,Producto A,30
3,2,Producto C,25
4,3,Producto A,10
5,3,Producto B,40
6,4,Producto A,25
7,4,Producto C,35
8,5,Producto B,20
9,5,Producto C,30


In [160]:
unidades_por_tienda

,ID_Tienda,Unidades_Vendidas
0,1,35
1,2,55
2,3,50
3,4,60
4,5,50


In [161]:
unidades_por_producto

,Producto,Unidades_Vendidas
0,Producto A,85
1,Producto B,75
2,Producto C,90


In [162]:
ventas['ventas_totales'] = ventas['Cantidad_Vendida']*ventas['Precio_Unitario']
ingresos_totales = ventas.groupby(['ID_Tienda'])['ventas_totales'].sum().reset_index().rename(columns={'ventas_totales': 'ingresos_totales_tienda'})
ingresos_totales

,ID_Tienda,ingresos_totales_tienda
0,1,5000
1,2,10500
2,3,9000
3,4,13000
4,5,13000


In [163]:
ventas['ventas_totales'].describe()

count       10.000000
mean      5050.000000
std       3361.960407
min       1000.000000
25%       2625.000000
50%       3500.000000
75%       7875.000000
max      10500.000000
Name: ventas_totales, dtype: float64

In [164]:
ventas_totales_tienda = ingresos_totales['ingresos_totales_tienda'].to_numpy()
mediana_np = np.median(ventas_totales_tienda)
desviacion_np = np.std(ventas_totales_tienda, ddof=1)
mediana_np, desviacion_np

(10500.0, 3324.1540277189324)

### Desempeño por categoría de producto

Se asignan categorías de ejemplo y se calcula el promedio de ventas por tienda y categoría para identificar preferencias.

In [165]:
categorias = pd.Series(["Deportes", "Hogar"])
inventarios['categoria'] = (categorias.sample(n=len(inventarios), replace=True, random_state=42).reset_index(drop=True))
inventarios

,ID_Tienda,Producto,Stock_Disponible,Fecha_Actualización,categoria
0,1,Producto A,50,2023-01-05,Deportes
1,1,Producto B,40,2023-01-06,Hogar
2,2,Producto A,60,2023-01-07,Deportes
3,2,Producto C,45,2023-01-08,Deportes
4,3,Producto A,30,2023-01-09,Deportes
5,3,Producto B,80,2023-01-10,Hogar
6,4,Producto C,70,2023-01-11,Deportes
7,4,Producto A,50,2023-01-12,Deportes
8,5,Producto B,40,2023-01-13,Deportes
9,5,Producto C,60,2023-01-14,Hogar


In [166]:
mezcla = ventas.merge(inventarios[['Producto', 'categoria']], on='Producto', how='left')
promedio = (
    mezcla.groupby(['ID_Tienda', 'categoria'])['Cantidad_Vendida']
         .mean()
         .reset_index()
         .rename(columns={'Cantidad_Vendida': 'promedio_ventas'})
)
promedio

,ID_Tienda,categoria,promedio_ventas
0,1,Deportes,19.000000
1,1,Hogar,15.000000
2,2,Deportes,28.333333
3,2,Hogar,25.000000
4,3,Deportes,16.000000
5,3,Hogar,40.000000
6,4,Deportes,28.333333
7,4,Hogar,35.000000
8,5,Deportes,26.666667
9,5,Hogar,23.333333


In [167]:
del inventarios['categoria']
inventarios

,ID_Tienda,Producto,Stock_Disponible,Fecha_Actualización
0,1,Producto A,50,2023-01-05
1,1,Producto B,40,2023-01-06
2,2,Producto A,60,2023-01-07
3,2,Producto C,45,2023-01-08
4,3,Producto A,30,2023-01-09
5,3,Producto B,80,2023-01-10
6,4,Producto C,70,2023-01-11
7,4,Producto A,50,2023-01-12
8,5,Producto B,40,2023-01-13
9,5,Producto C,60,2023-01-14


### Rotación de inventarios y niveles críticos

Se cruza la información de ventas con el stock disponible para estimar la rotación y detectar inventarios por debajo del 10%.

In [168]:
ventas_inventario = inventarios.merge(
    unidades_tienda_producto,
    on=['ID_Tienda', 'Producto'],
    how='left'
)
ventas_inventario['Unidades_Vendidas'] = ventas_inventario['Unidades_Vendidas'].fillna(0)
ventas_inventario['rotacion_inventario'] = np.where(
    ventas_inventario['Stock_Disponible'] > 0,
    ventas_inventario['Unidades_Vendidas'] / ventas_inventario['Stock_Disponible'],
    np.nan
)
ventas_inventario['porcentaje_vendido'] = ventas_inventario['rotacion_inventario'] * 100
ventas_inventario

,ID_Tienda,Producto,Stock_Disponible,Fecha_Actualización,Unidades_Vendidas,rotacion_inventario,porcentaje_vendido
0,1,Producto A,50,2023-01-05,20,0.400000,40.000000
1,1,Producto B,40,2023-01-06,15,0.375000,37.500000
2,2,Producto A,60,2023-01-07,30,0.500000,50.000000
3,2,Producto C,45,2023-01-08,25,0.555556,55.555556
4,3,Producto A,30,2023-01-09,10,0.333333,33.333333
5,3,Producto B,80,2023-01-10,40,0.500000,50.000000
6,4,Producto C,70,2023-01-11,35,0.500000,50.000000
7,4,Producto A,50,2023-01-12,25,0.500000,50.000000
8,5,Producto B,40,2023-01-13,20,0.500000,50.000000
9,5,Producto C,60,2023-01-14,30,0.500000,50.000000


In [169]:
inventario_critico = ventas_inventario[ventas_inventario['porcentaje_vendido'] < 10]
inventario_critico

,ID_Tienda,Producto,Stock_Disponible,Fecha_Actualización,Unidades_Vendidas,rotacion_inventario,porcentaje_vendido


### Satisfacción del cliente

Se analizan los niveles de satisfacción junto con los ingresos de cada tienda para priorizar acciones correctivas.

In [170]:
rendimiento_tiendas = ingresos_totales.merge(satisfaccion, on='ID_Tienda', how='left')
rendimiento_tiendas

,ID_Tienda,ingresos_totales_tienda,Satisfacción_Promedio,Fecha_Evaluación
0,1,5000,85,2023-01-15
1,2,10500,90,2023-01-15
2,3,9000,70,2023-01-15
3,4,13000,65,2023-01-15
4,5,13000,55,2023-01-15


In [171]:
tiendas_baja_satisfaccion = rendimiento_tiendas[rendimiento_tiendas['Satisfacción_Promedio'] < 60]
tiendas_baja_satisfaccion

,ID_Tienda,ingresos_totales_tienda,Satisfacción_Promedio,Fecha_Evaluación
4,5,13000,55,2023-01-15


### Estadísticos con NumPy

Se calculan la mediana y la desviación estándar de los ingresos totales utilizando NumPy para evidenciar la dispersión entre tiendas.

### Simulación de proyección de ventas

Se generan escenarios aleatorios de ventas futuras con una distribución normal basada en el desempeño histórico.

In [172]:
np.random.seed(42)
proyecciones = np.random.normal(
    loc=ventas_totales_tienda.mean(),
    scale=desviacion_np,
    size=(12, ventas_totales_tienda.size)
)
resumen_proyecciones = pd.DataFrame({
    'ID_Tienda': ingresos_totales['ID_Tienda'],
    'media_mensual_proyectada': proyecciones.mean(axis=0),
    'desviacion_proyectada': proyecciones.std(axis=0, ddof=1),
    'p95_proyectado': np.percentile(proyecciones, 95, axis=0)
})
resumen_proyecciones

,ID_Tienda,media_mensual_proyectada,desviacion_proyectada,p95_proyectado
0,1,10173.375030,2628.047777,13995.064224
1,2,9859.882296,3111.495504,15758.016747
2,3,10209.899666,2618.123243,13084.405883
3,4,8661.312123,3285.173227,13496.572034
4,5,9025.050947,3539.311305,13425.814792


### Conclusiones y próximos pasos

Las métricas permiten priorizar inventarios críticos y tiendas con baja satisfacción (<60%). Se recomienda profundizar en campañas locales y ajustar el stock según la proyección simulada.